# Description

This notebook computes expected coefficient values for all tissues (using their sample sizes) using random data.
These thresholds can be used to define "high" and "low" values.

# Modules

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import numpy.typing as npt
from scipy.spatial.distance import pdist

# Settings

In [2]:
N_GENES = 10000
N_JOBS = 10
RNG = np.random.RandomState(42)

In [3]:
GENERAL_PERCENTILES = np.linspace(0, 1, 101)[:-1]
display(GENERAL_PERCENTILES)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])

In [4]:
TOP_PERCENTILES = 1.0 - np.array([10 ** (-i) for i in range(3, 11)])
display(TOP_PERCENTILES)

array([0.999     , 0.9999    , 0.99999   , 0.999999  , 0.9999999 ,
       0.99999999, 1.        , 1.        ])

In [5]:
PERCENTILES = np.concatenate([GENERAL_PERCENTILES, TOP_PERCENTILES])
display(PERCENTILES)

array([0.        , 0.01      , 0.02      , 0.03      , 0.04      ,
       0.05      , 0.06      , 0.07      , 0.08      , 0.09      ,
       0.1       , 0.11      , 0.12      , 0.13      , 0.14      ,
       0.15      , 0.16      , 0.17      , 0.18      , 0.19      ,
       0.2       , 0.21      , 0.22      , 0.23      , 0.24      ,
       0.25      , 0.26      , 0.27      , 0.28      , 0.29      ,
       0.3       , 0.31      , 0.32      , 0.33      , 0.34      ,
       0.35      , 0.36      , 0.37      , 0.38      , 0.39      ,
       0.4       , 0.41      , 0.42      , 0.43      , 0.44      ,
       0.45      , 0.46      , 0.47      , 0.48      , 0.49      ,
       0.5       , 0.51      , 0.52      , 0.53      , 0.54      ,
       0.55      , 0.56      , 0.57      , 0.58      , 0.59      ,
       0.6       , 0.61      , 0.62      , 0.63      , 0.64      ,
       0.65      , 0.66      , 0.67      , 0.68      , 0.69      ,
       0.7       , 0.71      , 0.72      , 0.73      , 0.74   

In [6]:
print(f"Number of permutations is approximately: {N_GENES * (N_GENES - 1) / 2}")

Number of permutations is approximately: 49995000.0


In [7]:
# FIXME: hardcoded
TISSUE_NAMES = """
adipose_subcutaneous
adipose_visceral_omentum
adrenal_gland
artery_aorta
artery_coronary
artery_tibial
bladder
brain_amygdala
brain_anterior_cingulate_cortex_ba24
brain_caudate_basal_ganglia
brain_cerebellar_hemisphere
brain_cerebellum
brain_cortex
brain_frontal_cortex_ba9
brain_hippocampus
brain_hypothalamus
brain_nucleus_accumbens_basal_ganglia
brain_putamen_basal_ganglia
brain_spinal_cord_cervical_c1
brain_substantia_nigra
breast_mammary_tissue
cells_cultured_fibroblasts
cells_ebvtransformed_lymphocytes
cervix_ectocervix
cervix_endocervix
colon_sigmoid
colon_transverse
esophagus_gastroesophageal_junction
esophagus_mucosa
esophagus_muscularis
fallopian_tube
heart_atrial_appendage
heart_left_ventricle
kidney_cortex
kidney_medulla
liver
lung
minor_salivary_gland
muscle_skeletal
nerve_tibial
ovary
pancreas
pituitary
prostate
skin_not_sun_exposed_suprapubic
skin_sun_exposed_lower_leg
small_intestine_terminal_ileum
spleen
stomach
testis
thyroid
uterus
vagina
whole_blood
""".split()

In [8]:
assert len(TISSUE_NAMES) == 54

# Paths

In [13]:
BASE_DIR = Path("/home/miltondp/projects/ccc/ccc-gpu/ccc-gpu/base/")
GTEX_DIR = BASE_DIR / "data/gtex"

In [14]:
DATA_BY_TISSUE_PATH = GTEX_DIR / "gene_selection/all"
assert DATA_BY_TISSUE_PATH.exists()

In [15]:
OUTPUT_DIR = GTEX_DIR / "tissue_thresholds"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [16]:
OUTPUT_FILE_TEMPLATE = "{tissue}-null_coefs_percentiles.pkl"

# Functions

In [17]:
def pearson(data: pd.DataFrame) -> npt.NDArray:
    """
    Compute the Pearson correlation coefficient.

    Args:
        data: genes in rows, samples in columns.
    """
    return 1 - pdist(data.to_numpy(), metric="correlation")


def spearman(data: pd.DataFrame) -> npt.NDArray:
    """
    Compute the Spearman correlation coefficient.

    Args:
        data: genes in rows, samples in columns.
    """
    # compute ranks
    data = data.rank(axis=1)

    # corr_mat = 1 - pairwise_distances(data.to_numpy(), metric="correlation", n_jobs=1)
    return 1 - pdist(data.to_numpy(), metric="correlation")


def ccc(data: pd.DataFrame, internal_n_clusters=None, n_jobs=1) -> npt.NDArray:
    """
    Compute the Clustermatch Correlation Coefficient (CCC).

    Args:
        data: genes in rows, samples in columns.
    """
    # from ccc.coef import ccc
    from ccc.coef.impl_gpu import ccc

    return ccc(
        data.to_numpy(),
        internal_n_clusters=None,
        n_jobs=N_JOBS,
    )

# Compute

In [19]:
for tissue_name in TISSUE_NAMES:
    # get tissue data
    tissue_data = pd.read_pickle(
        DATA_BY_TISSUE_PATH / f"gtex_v8_data_{tissue_name}-var_pc_log2.pkl"
    )

    # select random set of genes
    tissue_data = tissue_data.sample(n=N_GENES, replace=False, axis=0, random_state=RNG)
    n_genes, n_samples = tissue_data.shape

    print(tissue_name, n_genes, n_samples, flush=False)

    # shuffle samples across genes
    data_shuffled = tissue_data.apply(
        lambda x: x.sample(frac=1, replace=False, random_state=RNG).to_numpy(), axis=0
    )

    tissue_null = pd.DataFrame(
        {
            "ccc": np.abs(ccc(data_shuffled)),
            "pearson": np.abs(pearson(data_shuffled)),
            "spearman": np.abs(spearman(data_shuffled)),
        }
    )

    tissue_null_perc = tissue_null.quantile(PERCENTILES)

    # save
    output_filename = OUTPUT_FILE_TEMPLATE.format(tissue=tissue_name)
    print(f"  {output_filename}", flush=True)
    tissue_null_perc.to_pickle(OUTPUT_DIR / output_filename)

adipose_subcutaneous 10000 663
  adipose_subcutaneous-null_coefs_percentiles.pkl
adipose_visceral_omentum 10000 541
  adipose_visceral_omentum-null_coefs_percentiles.pkl
adrenal_gland 10000 258
  adrenal_gland-null_coefs_percentiles.pkl
artery_aorta 10000 432
  artery_aorta-null_coefs_percentiles.pkl
artery_coronary 10000 240
  artery_coronary-null_coefs_percentiles.pkl
artery_tibial 10000 663
  artery_tibial-null_coefs_percentiles.pkl
bladder 10000 21
  bladder-null_coefs_percentiles.pkl
brain_amygdala 10000 152
  brain_amygdala-null_coefs_percentiles.pkl
brain_anterior_cingulate_cortex_ba24 10000 176
  brain_anterior_cingulate_cortex_ba24-null_coefs_percentiles.pkl
brain_caudate_basal_ganglia 10000 246
  brain_caudate_basal_ganglia-null_coefs_percentiles.pkl
brain_cerebellar_hemisphere 10000 215
  brain_cerebellar_hemisphere-null_coefs_percentiles.pkl
brain_cerebellum 10000 241
  brain_cerebellum-null_coefs_percentiles.pkl
brain_cortex 10000 255
  brain_cortex-null_coefs_percentiles.

# Testing

In [20]:
INPUT_FILE = OUTPUT_DIR / OUTPUT_FILE_TEMPLATE.format(tissue="adipose_subcutaneous")
display(INPUT_FILE)

PosixPath('/home/miltondp/projects/ccc/ccc-gpu/ccc-gpu/base/data/gtex/tissue_thresholds/adipose_subcutaneous-null_coefs_percentiles.pkl')

In [21]:
results = pd.read_pickle(INPUT_FILE)

In [22]:
results.shape

(108, 3)

In [23]:
results

,ccc,pearson,spearman
0.000000,0.000000,1.105982e-10,0.000000
0.010000,0.000000,3.110157e-04,0.000488
0.020000,0.000000,6.215730e-04,0.000976
0.030000,0.000000,9.297015e-04,0.001464
0.040000,0.000000,1.234677e-03,0.001953
...,...,...,...
0.999999,0.081933,9.932748e-01,0.189838
1.000000,0.089004,9.985866e-01,0.206378
1.000000,0.094901,9.995562e-01,0.211493
1.000000,0.095773,9.995769e-01,0.212342


In [24]:
results.describe()

,ccc,pearson,spearman
count,108.000000,1.080000e+02,108.000000
mean,0.013083,8.214961e-02,0.041910
std,0.021031,2.420337e-01,0.046971
min,0.000000,1.105982e-10,0.000000
25%,0.002061,6.288890e-03,0.013310
50%,0.005601,1.141103e-02,0.028447
75%,0.013642,2.052726e-02,0.050164
max,0.095860,9.995790e-01,0.212427
